In [ ]:
CREATE DATABASE ANANYA_SNOWFLAKE_;
CREATE SCHEMA DATA;

In [ ]:
CREATE OR REPLACE function text_chunker(pdf_text string)
returns table (chunk varchar)
language python
runtime_version = '3.9'
handler = 'text_chunker'
packages = ('snowflake-snowpark-python', 'langchain')
as
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

class text_chunker:

    def process(self, pdf_text: str):

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 1512, #Adjust this as you see fit
            chunk_overlap  = 256, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len
        )

        chunks = text_splitter.split_text(pdf_text)
        df = pd.DataFrame(chunks, columns=['chunks'])

        yield from df.itertuples(index=False, name=None)
$$;

In [ ]:
CREATE OR REPLACE stage docs DIRECTORY = ( ENABLE = true )
    ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
CREATE OR REPLACE TABLE DOCS_CHUNKS_TABLE (
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    SIZE NUMBER(38,0), -- Size of the PDF
    FILE_URL VARCHAR(16777216), -- URL for the PDF
    SCOPED_FILE_URL VARCHAR(16777216), -- Scoped url (you can choose which one to keep depending on your use case)
    CHUNK VARCHAR(16777216), -- Piece of text
    CATEGORY VARCHAR(16777216) -- Will hold the document category to enable filtering
);

In [ ]:
INSERT INTO docs_chunks_table (relative_path, size, file_url,
                            scoped_file_url, chunk)

    SELECT relative_path,
            size,
            file_url,
            build_scoped_file_url(@docs, relative_path) AS scoped_file_url,
            func.chunk AS chunk
    FROM
        directory(@docs),
        TABLE(text_chunker (TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@docs,
                              relative_path, {'mode': 'LAYOUT'})))) AS func;

In [ ]:
CREATE
OR REPLACE TABLE docs_categories AS (
  SELECT
    relative_path, file_url, chunk, size
  FROM
    docs_chunks_table
);

In [ ]:
CREATE OR REPLACE TABLE docs_categories_final AS (
SELECT
    relative_path,
    file_url,
    chunk, size,
    snowflake.cortex.COMPLETE(
        'llama3-70b',
        'Categorize the document if it is related to "Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona",
        "California", "Colorado", "Connecticut", "District of Columbia",
        "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa",
        "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana",
        "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota",
        "Missouri", "Mississippi", "Montana", "North Carolina",
        "North Dakota", "Nebraska", "New Hampshire", "New Jersey",
        "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon",
        "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina",
        "South Dakota", "Tennessee", "Texas", "Utah", "Virginia",
        "Virgin Islands", "Vermont", "Washington", "Wisconsin",
        "West Virginia", "Wyoming". Use only one word: <file> ' || relative_path || '</file>'
    ) AS test_output
FROM
    docs_chunks_table
);

In [ ]:
CREATE or REPLACE CORTEX SEARCH SERVICE CC_SEARCH_SERVICE_CS_AN
ON chunk
ATTRIBUTES TEST_OUTPUT
warehouse = COMPUTE_WH
TARGET_LAG = '1 minute'
as (
    SELECT chunk,
        relative_path,
        file_url,
        TEST_OUTPUT
    FROM docs_categories_final
);